funkcja która jako argument dostaje wartości parametrów a wyrzuca ocenę klasyfikatora


In [16]:
def quality_for_given_n_mfcc (n_mfcc=20, train_size=40, test_size=10, k=5):
    # 1. Ekstrakcja cech z próbek
    classes = []
    class_size = 100 # użyjemy tylko po kilka danych
    samples = []
    labels = []

    for command in commands:
        classes.append(command)
        sample_num = 0
        for file in os.listdir(train_audio_path + '/' + command + '/'):
            sample_num += 1
            y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
            mfcc_seq = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = n_mfcc)
            mean_mfcc_seq = []
            for feature in mfcc_seq:
                mean_mfcc_seq.append(numpy.mean(feature))
            samples.append(mean_mfcc_seq)
            labels.append(command)
            if sample_num == class_size:
                break
    
    # 2. Wybór danych treningowych i testowych

    train_samples = []
    train_labels = []
    test_samples = []
    test_labels = []
    for classname in classes:
        train_size_index = 0
        test_size_index = 0
        for i in range(len(labels)):
            if labels[i] == classname and train_size_index < train_size:
                train_samples.append(samples[i])
                train_labels.append(labels[i])
                train_size_index += 1
            elif labels[i] == classname and test_size_index < test_size:
                test_samples.append(samples[i])
                test_labels.append(labels[i])
                test_size_index += 1
            if train_size_index == train_size and test_size_index == test_size:
                break
    
    # 3. Uczenie klasyfikatora danymi treningowymi
    # skalowanie/normalizacja danych
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(train_samples)
    train_samples = scaler.transform(train_samples)
    test_samples = scaler.transform(test_samples)

    # uczenie
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = k)
    classifier.fit(train_samples, train_labels)
    
    # 4. Klasyfikacja nieznanych próbek
    predicted_labels = classifier.predict(test_samples)
    
    # 5. Ocena klasyfikatora
    quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
    
    return quality
    

In [1]:
# import bibliotek
import os
import librosa
import pickle
import sklearn
import numpy

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'input/speech_commands_v0.01'
commands = os.listdir(train_audio_path) 
commands=commands[1:15]


1. Znalezienie optymalnej wartości parametru n_mfcc

In [8]:
n_mfcc_values=[]
qualities=[]
for i in range (1, 100):
    quality = quality_for_given_n_mfcc(n_mfcc=i, train_size=40, test_size=10, k=5)
    n_mfcc_values.append(i)
    qualities.append(quality)
    print('Dla n_mfc c= %d Ocena klasyfikatora: %.2f %%' % (i, quality))

Dla n_mfcc= 1 Ocena klasyfikatora: 1.43 %
Dla n_mfcc= 2 Ocena klasyfikatora: 4.29 %
Dla n_mfcc= 3 Ocena klasyfikatora: 11.43 %
Dla n_mfcc= 4 Ocena klasyfikatora: 8.57 %
Dla n_mfcc= 5 Ocena klasyfikatora: 17.14 %
Dla n_mfcc= 6 Ocena klasyfikatora: 17.14 %
Dla n_mfcc= 7 Ocena klasyfikatora: 17.14 %
Dla n_mfcc= 8 Ocena klasyfikatora: 17.14 %
Dla n_mfcc= 9 Ocena klasyfikatora: 12.86 %
Dla n_mfcc= 10 Ocena klasyfikatora: 12.86 %
Dla n_mfcc= 11 Ocena klasyfikatora: 15.71 %
Dla n_mfcc= 12 Ocena klasyfikatora: 15.71 %
Dla n_mfcc= 13 Ocena klasyfikatora: 12.86 %
Dla n_mfcc= 14 Ocena klasyfikatora: 14.29 %
Dla n_mfcc= 15 Ocena klasyfikatora: 17.14 %
Dla n_mfcc= 16 Ocena klasyfikatora: 12.86 %
Dla n_mfcc= 17 Ocena klasyfikatora: 17.14 %
Dla n_mfcc= 18 Ocena klasyfikatora: 17.14 %
Dla n_mfcc= 19 Ocena klasyfikatora: 18.57 %
Dla n_mfcc= 20 Ocena klasyfikatora: 20.00 %
Dla n_mfcc= 21 Ocena klasyfikatora: 18.57 %
Dla n_mfcc= 22 Ocena klasyfikatora: 15.71 %
Dla n_mfcc= 23 Ocena klasyfikatora: 18.57 %


Optymalna wartość n_mfcc:

In [17]:
n_mfcc_optimal = n_mfcc_values[qualities.index(max(qualities))]
print(n_mfcc_optimal)

20


2. Znalezienie optymalnej wartości train size

In [ ]:
train_size_values=[]
qualities_train=[]
for i in range (1, 90):
    quality = quality_for_given_n_mfcc(n_mfcc=n_mfcc_optimal, train_size=i, test_size=10, k=5)
    train_size_values.append(i)
    qualities_train.append(quality)
    print('Dla train_size = %d Ocena klasyfikatora: %.2f %%' % (i, quality))

Optymalna wartość train_size:

In [ ]:
train_size_optimal = train_size_values[qualities_train.index(max(qualities_train))]
print(test_size_optimal)

3. Znalezienie optymalnej wartości k

In [ ]:
k_values=[]
qualities_k=[]
for i in range (1,11):
    quality = quality_for_given_n_mfcc(n_mfcc=n_mfcc_optimal, train_size=train_size_optimal, test_size=10, k=i)
    k_values.append(i)
    qualities_k.append(quality)
    print('Dla k = %d Ocena klasyfikatora: %.2f %%' % (i, quality))

Optymalna wartość k :

In [ ]:
k_optimal = k_values[qualities_k.index(max(qualities_k))]
print(k_optimal)